<img src='../../img/anaconda-logo.png' align='left' style="padding:10px">
<br>
*Copyright Continuum 2012-2016 All Rights Reserved.*

# Datashader & BokehGeo: GIS Raster Data

In this lesson, we'll see how to use Datashader and Bokeh together to create interactive GIS Visualizations.

## Table of Contents
* [Datashader & BokehGeo: GIS Raster Data](#Datashader-&-BokehGeo:-GIS-Raster-Data)
	* [Set-up](#Set-up)
* [Preparations](#Preparations)
	* [Projections](#Projections)
	* [Datashader Transfer Functions](#Datashader-Transfer-Functions)
	* [Adding raster with datashader](#Adding-raster-with-datashader)
* [Using BokehGeo Transfer Functions with Datashader](#Using-BokehGeo-Transfer-Functions-with-Datashader)
	* [Slope](#Slope)
	* [Aspect](#Aspect)
	* [Slope-Aspect Map: Combining multiple aggregates](#Slope-Aspect-Map:-Combining-multiple-aggregates)
	* [Hillshading](#Hillshading)
	* [NDVI](#NDVI)


## Set-up

Test versions

In [ ]:
import bokeh, datashader
print( bokeh.__version__ )
print( datashader.__version__ )

Upgrade if needed ...

In [ ]:
# !anaconda config --set default_site binstar
# !conda install datashader=0.3.2 -y  # should be using 0.3.2
# !anaconda config --set default_site aws

The following imports will be needed to complete the exercises or provide for an improved notebook display:

In [ ]:
import numpy as np
import rasterio as rio

from bokeh.models import Range1d
from bokeh.plotting import Figure
from bokeh.io import output_notebook, show

from bokeh.resources import INLINE
from datashader.colors import Hot

from bokeh.tile_providers import STAMEN_TONER

from pyproj import transform, Proj

output_notebook()

# Preparations

## Projections

The `pyproj` library is a great tool for projections. In this section, we'll see how to use it.

When converting between coordinate systems using `pyproj`, start by creating a `Proj` object for `input` and `output` coordinate systems.

In [ ]:
from pyproj import transform, Proj

input_proj = Proj(init='epsg:4326') # Geographic Projection (decimal degrees)
output_proj = Proj(init='epsg:3857') # Web Mercator Projection (meters)

pass longitude and latitude into `pyproj.transform` to convert between coordindate systems

In [ ]:
xmin_lng = -139
xmax_lng = -50
ymin_lat = 20
ymax_lat = 55

xmin_meters, ymin_meters = transform(input_proj, output_proj, xmin_lng, ymin_lat)
xmax_meters, ymax_meters = transform(input_proj, output_proj, xmax_lng, ymax_lat)

Instead of having to call transform for each coordinate pair, we can vectorize the coordinate system projection using numpy arrays:

In [ ]:
# reprojecting can be vectorized
x_longitudes = np.array([-139, -50])
y_latitudes = np.array([20, 55])
x_meters, y_meters = transform(input_proj, output_proj, x_longitudes, y_latitudes)

In [ ]:
# Bokeh 1D Ranges
austin_x_range = Range1d(*x_meters)
austin_y_range = Range1d(*y_meters)

## Datashader Transfer Functions

In [ ]:
def base_plot(tools='pan,wheel_zoom,reset',plot_width=900, plot_height=600, x_range=None, y_range=None, **plot_args):
    p = Figure(tools=tools, plot_width=plot_width, plot_height=plot_height,
        x_range=x_range, y_range=y_range, outline_line_color=None,
        min_border=0, min_border_left=0, min_border_right=0,
        min_border_top=0, min_border_bottom=0, **plot_args)
    
    p.axis.visible = False
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    p.add_tile(STAMEN_TONER, alpha=.5)
    return p

Loading a raster into a datashader canvas

In [ ]:
# load austin elevation data
path = '../../data/Datashader/austin_dem.tif'
raster_data = rio.open(path)

# define the extent of the canvas to be used such that all data in `austin_dem.tif` fits within the canvas.
xmin = -11020645
ymin = 3503546
xmax = -10797986
ymax = 3632767

## Adding raster with datashader

For this demo we will use a helper function `InteractiveImage`, which takes both a plot and and a function, and provides interactivity within this jupyter notebook.

In [ ]:
from datashader.bokeh_ext import InteractiveImage

import datashader as ds
import datashader.transfer_functions as tf

The basics using `Canvas.raster`

`ds.Canvas.raster(data)` resamples the raster so that the resolution and allignment "fit" inside your canvas scene in a uniform way so that as you interact and change the scene, the aggregation ("resampling" in this case, since the data is already gridded) is recomputed every time you move or zoom on the image.

In [ ]:
def basic_raster(x_range, y_range, w, h, how='log'):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    agg = cvs.raster(raster_data)
    return tf.interpolate(agg, cmap=['darkred','white','darkblue'], how='linear', alpha=255)

p = base_plot(x_range=(-11020645, -10797986), y_range=(3503546, 3632767))
InteractiveImage(p, basic_raster)

# Using BokehGeo Transfer Functions with Datashader

`bokeh_geo.transfer_functions` contains many transfer functions useful together with Datashader. Here we demonstrate all 5 of them.

* slope
* aspect
* hillshade
* ndvi


We will also demonstrate using two together:
* slow & aspect

## Slope

Now apply a transfer function `slope()` from BokehGeo. And use it with DataShader.

In [ ]:
from bokeh_geo.transfer_functions import slope

def slope_raster(x_range, y_range, w, h, how='log'):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    agg = cvs.raster(raster_data)
    slope_agg = slope(agg)
    return tf.interpolate(slope_agg, cmap=['lightgray','purple'], how='eq_hist')

p = base_plot(x_range=(-11020645, -10797986), y_range=(3503546, 3632767))  ## values from cell above
InteractiveImage(p, slope_raster)

## Aspect

Aspect, in context of GIS, is the direction of downward slope measured from 0-359 degrees.

Here we demonstrate the use of the BokehGeo `transfer_function` sub-module, importing `aspect` and using it with DataShader.

In [ ]:
from bokeh_geo.transfer_functions import aspect

from datashader.bokeh_ext import HoverLayer

def aspect_raster(x_range, y_range, w, h, how='log'):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    agg = cvs.raster(raster_data)
    aspect_agg = aspect(agg)
    return tf.interpolate(aspect_agg, cmap=['aqua', 'black'], how='eq_hist')

p = base_plot(x_range=(-11020645, -10797986), y_range=(3503546, 3632767))
InteractiveImage(p, aspect_raster)

## Slope-Aspect Map: Combining multiple aggregates

In this example, we demonstrate how you can use two aggregates together on one canvas.

In the example:
- load slope and aspect aggregates
- reclassify slope into 4 classes with values of (10, 20, 30, 40)
- reclassify aspect into 8 classes with values of (1, 2, 3, 4, 5, 6, 7, 8)
- sum the two reclassified aggregates together
- pixels with value less than 20 are consider background and are colored gray
- other pixels are color based on their slope and aspect values

In [ ]:
from bokeh_geo.transfer_functions import slope, aspect,color_values 

def slope_aspect_raster(x_range, y_range, w, h, how='log'):
    '''
    inspired by http://www.personal.psu.edu/cab38/Terrain/AutoCarto.html
    '''
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    agg = cvs.raster(raster_data)
    
    slope_agg = aspect(agg)
    slope_agg = slope(agg) * 100
    bins = [0, 5, 20, 40]
    slope_agg.data = np.digitize(slope_agg.data, bins) * 10
    
    aspect_agg = aspect(agg)
    bins = [0, 22, 67, 112, 157, 202, 247, 292, 337]
    aspect_agg.data = np.digitize(aspect_agg.data, bins)
    aspect_agg.data[aspect_agg.data == 9] = 1
    
    agg_sum = slope_agg + aspect_agg
    
    background = (153, 153, 153)
    
    final_colors = {
        11 : background,
        12 : background,
        13 : background,
        14 : background,
        15 : background,
        16 : background,
        17 : background,
        18 : background,
        19 : background,
        21 : (147,166,89),
        22 : (102,153,102),
        23 : (102,153,136),
        24 : (89,89,166),
        25 : (128,108,147),
        26 : (166,89,89),
        27 : (166,134,89),
        28 : (166,166,89),
        31 : (172,217,38),
        32 : (77,179,77),
        33 : (73,182,146),
        34 : (51,51,204),
        35 : (128,89,166),
        36 : (217,38,38),
        37 : (217,142,38),
        38 : (217,217,38),
        41 : (191,255,0),
        42 : (51,204,51),
        43 : (51,204,153),
        44 : (26,26,230),
        45 : (128,51,204),
        46 : (255,0,0),
        47 : (255,149,0),
        48 : (255,255,0)
    }
 
    return color_values(agg_sum, final_colors)

p = base_plot(x_range=(-11020645, -10797986), y_range=(3503546, 3632767))
InteractiveImage(p, slope_aspect_raster)

## Hillshading

Hillshading is about adding pseudo-relief based on the angle and altitude of a light source. 

In [ ]:
from bokeh_geo.transfer_functions import hillshade
from bokeh_geo.colors import elevation_ramp

from datashader.bokeh_ext import HoverLayer

def hillshade_raster(x_range, y_range, w, h, how='log'):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    agg = cvs.raster(raster_data)
    return tf.stack(tf.interpolate(agg, cmap=['#333333', 'white'], how='eq_hist', alpha=255),
                          hillshade(agg, how='mdow', alpha=45))

p = base_plot(x_range=(-11020645, -10797986), y_range=(3503546, 3632767))
InteractiveImage(p, hillshade_raster)

## NDVI

NVDI = "Normalized Difference Vegetation Index"

Relative difference, rescaling to the range of values.

Load two images

* near IR light (NIR)
* Red light

Context

* We know vegetation absorbs a lot of red, and reflects infra-red
* So we look for high absorption of red and low absorption of IR, and assume that is a proxy for plant life.
* Plants look green, so the red absorption is intuitive, but the IR piece is not so intuitive.


In [ ]:
from bokeh_geo.transfer_functions import ndvi
from datashader.bokeh_ext import InteractiveImage

nir_data = rio.open('../../data/Datashader/roswell_landsat8_b5_nir_mercator.tif')
red_data = rio.open('../../data/Datashader/roswell_landsat8_b4_red_mercator.tif')

xmin = nir_data.bounds.left
ymin = nir_data.bounds.bottom
xmax = nir_data.bounds.right
ymax = nir_data.bounds.top

In [ ]:
def ndvi_raster(x_range, y_range, w, h, how='log'):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    nir_agg = cvs.raster(nir_data)
    red_agg = cvs.raster(red_data)
    ndvi_agg = ndvi(nir_agg, red_agg)
    return tf.interpolate(ndvi_agg, cmap=['magenta', 'black', 'limegreen'], how='linear')

p = base_plot(x_range=(xmin, xmax), y_range=(ymin, ymax))
InteractiveImage(p, ndvi_raster)

---
*Copyright Continuum 2012-2016 All Rights Reserved.*